First Jupyter Notebook For Debugging Purposes!

In [1]:
using Plots

# Constants: (SI Units)
const M_S = 1.989e30 # The Mass of The Sun (Kg)

1.989e30

In [2]:
const M_E = 5.972e24 # The Mass of The Earth (Kg)

5.972e24

In [3]:
const M_M = 7.34767309e22 # The Mass of The Moon (Kg)

7.34767309e22

In [5]:
const G = 6.6743e-11/(AU^3) # Gravitational Constant (m^3kg^-1s^-2)

1.8793249973671362e-29

In [6]:
const SEMD = AU # Sun-Earth Mean Distance = 149597870.700 Km

1

In [7]:
const EMMD = AU / 391.1055443137255 # Earth-Moon Mean Distance = 382,500 Km

0.0025568545742676806

In [8]:
const M_i = 5.15 # The Moon moves in an approximately elliptic orbit inclined 
# at about five degrees to the plane of the ecliptic. (In Degrees)

5.15

In [9]:
## Linear Velocity of The Earth In Orbit Around The Sun:
# This equation can be proved with the help of polar
# coordinates and two assumptions:
# 1- r is constant in orbital motion (r = AU)
# 2- Theta_Dot is constant in orbital motuon (Theta_Dot = cte.)
const V_E0 = (G * M_S / SEMD)^0.5 # Earth Initial Velocity

6.1139000807694215

In [10]:
const V_M0 = (G * M_E / EMMD)^0.5 + V_E0 # Moon Initial Velocity

6.323411563803095

In [11]:
# State-Space Matrix Calculation Function:
function f(x_var)
    D_A1_A2 = 1 / (((x_var[1] - x_var[4])^2 + (x_var[2] - x_var[5])^2 + (x_var[3] - x_var[6])^2)^(3 / 2))
    D_B1 = 1 / ((x_var[1]^2 + x_var[2]^2 + x_var[3]^2)^(3 / 2)) # Inverse of The Distance of The Moon
    D_B2 = 1 / ((x_var[4]^2 + x_var[5]^2 + x_var[6]^2)^(3 / 2)) # Inverse of The Distance of The Earth

    A1 = (G * M_E) / D_A1_A2
    A2 = (G * M_M) / D_A1_A2
    B1 = (G * M_S) / D_B1
    B2 = (G * M_S) / D_B2

    Z1 = -B1 - A1
    Z2 = -B2 - A2

    # Stat-Space Matrix
    A = [0 0 0 0 0 0 1 0 0 0 0 0
        0 0 0 0 0 0 0 1 0 0 0 0
        0 0 0 0 0 0 0 0 1 0 0 0
        0 0 0 0 0 0 0 0 0 1 0 0
        0 0 0 0 0 0 0 0 0 0 1 0
        0 0 0 0 0 0 0 0 0 0 0 1
        Z1 0 0 A1 0 0 0 0 0 0 0 0
        0 Z1 0 0 A1 0 0 0 0 0 0 0
        0 0 Z1 0 0 A1 0 0 0 0 0 0
        A2 0 0 Z2 0 0 0 0 0 0 0 0
        0 A2 0 0 Z2 0 0 0 0 0 0 0
        0 0 A2 0 0 Z2 0 0 0 0 0 0]

    f_x = A * x_var
    return f_x
end

f (generic function with 1 method)

In [12]:
# Initialize System:
Day_Hours = 23 + (56 / 60) + (4 / 3600) # Hours of A Day

23.934444444444445

In [13]:
Year_Days = 365.2425 # Days of 

365.2425

In [14]:
Year_Seconds = Year_Days * Day_Hours * 3600

3.1470754770000003e7

In [15]:
dD = 0.1 # Simulation Step In Day

0.1

In [16]:
Stop_Year = 0.08 # The year that the simulation will stop.

0.08

In [17]:
dY = dD / 365.2425 # Simulation  Step In Years

0.00027379070069885077

In [18]:
Δt = dY * Year_Seconds # In Seconds

8616.400000000001

In [19]:
Year = 0:dY:Stop_Year # In Years

0.0:0.00027379070069885077:0.07994688460406442

In [20]:
xM0 = -SEMD - (EMMD * cosd(M_i))

-1.0025465328367962

In [21]:
yM0 = 0

0

In [22]:
zM0 = EMMD * sind(M_i)

0.0002295121458045649

In [23]:
VxM0 = 0

0

In [24]:
VyM0 = -V_M0

-6.323411563803095

In [25]:
VzM0 = 0

0

In [26]:
xE0 = -SEMD

-1

In [27]:
yE0 = 0

0

In [28]:
zE0 = 0

0

In [29]:
VxE0 = 0

0

In [30]:
VyE0 = -V_E0

-6.1139000807694215

In [31]:
VzE0 = 0

0

In [32]:
x = zeros(12, length(Year));

In [33]:
x[:, 1] = [xM0 yM0 zM0 xE0 yE0 zE0 VxM0 VyM0 VzM0 VxE0 VyE0 VzE0]'

12×1 adjoint(::Matrix{Float64}) with eltype Float64:
 -1.0025465328367962
  0.0
  0.0002295121458045649
 -1.0
  0.0
  0.0
  0.0
 -6.323411563803095
  0.0
  0.0
 -6.1139000807694215
  0.0

In [34]:
i = 1

1

In [35]:
x[:, i+1] = x[:, i] + Δt .* f(x[:, i])

12-element Vector{Float64}:
     -1.0025465328367962
 -54485.043398352995
      0.0002295121458045649
     -1.0
 -52679.808655941655
      0.0
 325372.40490947117
     -6.323411563803095
    -74.48723464741639
 322079.08639647934
     -6.1139000807694215
      4.564585263400823e-14

In [ ]:
k3 = Δt .* f(x[:, i] + k2 ./ 2)